## Zadania: Model agnostic gateway production


Zadanie 1 \
Routing po nagłówku X-Model \
W tym zadaniu symulujemy wybór modelu z rejestru modeli po nagłówku HTTP.

In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict, Any

def openai_gpt(prompt: str) -> str: return f"[openai:gpt] {prompt[:40]}..."
def bedrock_claude(prompt: str) -> str: return f"[bedrock:claude] {prompt[:40]}..."
____ # <- dodaj analogiczną funkcję dla nowego modelu jak powyżej

REGISTRY: Dict[str, Any] = {
    "openai:gpt-5-mini": openai_gpt,
    "aws:claude-3-7": bedrock_claude,
    "____": ____ # <- tu wstaw dodatkowy model
}

@dataclass
class Ctx:
    model: Optional[str]
    prompt: str

def pick_model(ctx: Ctx):
    # TODO: wybierz funkcję z REGISTRY na podstawie ctx.model
    return REGISTRY.get(ctx.model or "", REGISTRY["openai:gpt-5-mini"])

fn = pick_model(Ctx("aws:claude-3-7","Hello"))
print(fn("Test"))

Zadanie 2 \
Fallback + log kosztu \
Korzystając z kodu przygortowanego w poprzednim zadniu uzupełnij ponizszy kod o koszty dla nowo dodanego modelu.

In [ ]:
COSTS = {
    "openai:gpt-5-mini":0.0003,
    "aws:claude-3-7":0.0005,
    "____":____
}
def run(ctx: Ctx):
    try:
        fn = pick_model(ctx)
        out = fn(ctx.prompt)
        key = ctx.model if ctx.model in REGISTRY else "openai:gpt-5-mini"
    except Exception:
        out = REGISTRY["openai:gpt-5-mini"](ctx.prompt)  # TODO: fallback
        key = "openai:gpt-5-mini"
    log = {"model": key, "est_cost_per_1k": COSTS.get(key,0.0)}  # TODO: realnie po tokenach
    return out, log
print(run(Ctx("____","Krótka odpowiedź o Pythonie."))) # <- wstaw klucz wybranego modelu w rejestrze

Zadanie 3 \
Human-in-the-Loop (approve/edit/reject) \
Uzupełnij "____" w poniższym kodzie, aby zasymulować obsługę Human in the Loop.

In [ ]:
from enum import Enum
class Decision(str, Enum):
    approve="approve"
    edit="edit"
    reject="reject"

def hitl(output: str, decision: Decision, edited: str|None=None) -> str:
    # TODO: jeśli approve->output; edit->edited; reject->komunikat o odrzuceniu
    if decision==Decision.____:  # <- wstaw pasujace opcje Decision do zwracanych wartości
        return output
    if decision==Decision.____ and edited:
        return edited
    if decision==Decision.____:
        return "[REJECTED] Do poprawy."
    return output

final = hitl(*run(Ctx("openai:gpt-5-mini","Opisz Python w 1 zdaniu."))[0:1], decision=Decision.edit)  # <- Zmień Decision.edit kolejno na pozostałe opcje i uruchom ponownie kod
print(final)